In [ ]:
from numpy import sqrt, rint, zeros, random, zeros, ones

In [ ]:
N=10

In [ ]:
rx=zeros(N)

In [ ]:
rx=random.normal(0.5, 2, N)

In [ ]:
rx

In [ ]:
r2max=1.

In [ ]:
for k in range(N-1) :
    j=k+1
    #for j in range(k+1,N) :
    dx = rx[k]-rx[j:N]
    print(dx)
    input("Press Enter to continue...")
    dx[...]-= rint(dx)
    print(dx)
    input("Press Enter to continue...")
    r2 = dx*dx
    # using the mask array "b" for speedup
    b = r2 < r2max
    print(b)
    input("Press Enter to continue...")
    lm  = sqrt(r2[b])
    print('lm=', lm)
    #if lm<self.kg :
    for elm in lm :
        print(elm)

In [ ]:
N

In [ ]:
npmax=5
from numpy import cos, sin, pi

In [ ]:
cp = zeros((2*npmax+1,N))
cp[npmax,] = 1.0
sp = zeros((2*npmax+1,N))


In [ ]:
cp

In [ ]:
sp

In [ ]:
cp[npmax+1,] = cos(2*pi*rx)
sp[npmax+1,] = sin(2*pi*rx)

In [ ]:
cp

In [ ]:
sp

In [ ]:
for n in range(npmax-1):
    m = npmax+2+n
    print(n, m)
    cp[m,] = cp[npmax+1,]*cp[m-1,]-sp[npmax+1,]*sp[m-1,]
    sp[m,] = sp[npmax+1,]*cp[m-1,]+cp[npmax+1,]*sp[m-1,]

In [ ]:
cp

In [ ]:
sp

In [ ]:
for n in range(1,npmax+1):
    mm = npmax-n
    mp = npmax+n
    print(mp,mm)
    cp[mm,] = cp[mp,]
    sp[mm,] = -sp[mp,]

In [ ]:
cp

In [ ]:
sp

In [12]:
from numba import jit
import numpy as np
import math
# import  zeros, rint, cos, sin, ones, pi


# @jit(nopython=True, cache=True)
# @jit
def calcsofk(rx, ry, rz, nmax):

    N = rx.size

    cp = np.zeros((3, 2 * nmax + 1, N))
    sp = np.zeros((3, 2 * nmax + 1, N))
    cp[:, nmax, :] = 1.0     # this is the k=0 vector
    # these are the contribution of the first k-vector (1,0,0) and symmetries.
    cp[0, nmax+1, :] = np.cos(2 * np.pi * rx[:])
    cp[1, nmax+1, :] = np.cos(2 * np.pi * ry[:])
    cp[2, nmax+1, :] = np.cos(2 * np.pi * rz[:])
    sp[0, nmax+1, :] = np.sin(2 * np.pi * rx[:])
    sp[1, nmax+1, :] = np.sin(2 * np.pi * ry[:])
    sp[2, nmax+1, :] = np.sin(2 * np.pi * rz[:])
    #
    # for n in range(nmax-1):
    for n in range(nmax):
        m = nmax + 1 + n
        cp[:, m, :] = (cp[:, nmax+1, :] * cp[:, m-1, :] -
                       sp[:, nmax+1, :] * sp[:, m-1, :])
        sp[:, m, :] = (sp[:, nmax+1, :] * cp[:, m-1, :] +
                       cp[:, nmax+1, :] * sp[:, m-1, :])
    for n in range(1, nmax+1):
        mm = nmax-n
        mp = nmax+n
        # print(mp,mm)
        # impose the symmetry of cos and sin along the axes
        cp[:, mm, :] = cp[:, mp, :]
        sp[:, mm, :] = - sp[:, mp, :]

    # these are the sum of cos and sin over particles.
    # indices are along the three directions (nx, ny, nz)
    csum = np.zeros((2*nmax+1, 2*nmax+1, 2*nmax+1), dtype=float)
    ssum = np.zeros((2*nmax+1, 2*nmax+1, 2*nmax+1), dtype=float)

    for k in range(N):
        for nx in range(2 * nmax + 1):
            for ny in range(2 * nmax + 1):
                for nz in range(2 * nmax + 1):
                    csum[nx, ny, nz] += (cp[0, nx, k] * cp[1, ny, k] * cp[2, nz, k] -
                                         cp[0, nx, k] * sp[1, ny, k] * sp[2, nz, k] -
                                         sp[0, nx, k] * sp[1, ny, k] * cp[2, nz, k] -
                                         sp[0, nx, k] * cp[1, ny, k] * sp[2, nz, k])
                    ssum[nx, ny, nz] += (sp[0, nx, k] * cp[1, ny, k] * cp[2, nz, k] -
                                         sp[0, nx, k] * sp[1, ny, k] * sp[2, nz, k] -
                                         cp[0, nx, k] * sp[1, ny, k] * cp[2, nz, k] +
                                         cp[0, nx, k] * cp[1, ny, k] * cp[2, nz, k])

    sofk = np.zeros((2*nmax+1, 2*nmax+1, 2*nmax+1), dtype=float)
    sofk = (csum**2 + ssum**2) / N
    return sofk
    #return (csum**2 + ssum**2) / N

if __name__ == '__main__':

    rx = np.asarray([1., 0.0, 0.0])
    ry = np.asarray([0.0, 1.0, 0.0])
    rz = np.asarray([0.0, 0.0, 1.0])
    nmax=3
    
    sofk=calcsofk(rx,ry,rz,nmax)
    for nx in range(2 * nmax + 1):
        for ny in range(2 * nmax + 1):
            for nz in range(2 * nmax + 1):
                print(nx-nmax,ny-nmax,nz-nmax,(nx-nmax)**2+(ny-nmax)**2+(nz-nmax)**2,sofk[nx,ny,nz])
    
#    print('Sk:', calcsofk(rx, ry, rz, 4))

-3 -3 -3 27 6.0
-3 -3 -2 22 6.0
-3 -3 -1 19 6.0
-3 -3 0 18 6.0
-3 -3 1 19 6.0
-3 -3 2 22 6.0
-3 -3 3 27 6.0
-3 -2 -3 22 6.0
-3 -2 -2 17 6.0
-3 -2 -1 14 6.0
-3 -2 0 13 6.0
-3 -2 1 14 6.0
-3 -2 2 17 6.0
-3 -2 3 22 6.0
-3 -1 -3 19 6.000000000000001
-3 -1 -2 14 6.000000000000001
-3 -1 -1 11 6.000000000000001
-3 -1 0 10 6.000000000000001
-3 -1 1 11 6.000000000000001
-3 -1 2 14 6.000000000000001
-3 -1 3 19 6.000000000000001
-3 0 -3 18 6.000000000000003
-3 0 -2 13 6.000000000000003
-3 0 -1 10 6.000000000000003
-3 0 0 9 6.000000000000003
-3 0 1 10 6.000000000000003
-3 0 2 13 6.000000000000003
-3 0 3 18 6.000000000000003
-3 1 -3 19 6.000000000000003
-3 1 -2 14 6.000000000000003
-3 1 -1 11 6.000000000000003
-3 1 0 10 6.000000000000003
-3 1 1 11 6.000000000000003
-3 1 2 14 6.000000000000003
-3 1 3 19 6.000000000000003
-3 2 -3 22 6.000000000000003
-3 2 -2 17 6.000000000000003
-3 2 -1 14 6.000000000000003
-3 2 0 13 6.000000000000003
-3 2 1 14 6.000000000000003
-3 2 2 17 6.000000000000003
-3 2 3 22 

In [10]:
sofk.shape

(9, 9, 9)